# Logistic Regression - Problem Statement

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [0]:
data = spark.read.csv('/FileStore/tables/customer_churn.csv',inferSchema=True,
                     header=True)

In [0]:
data.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: string (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- Churn: integer (nullable = true)

### Check out the data

In [0]:
data.describe().show()

+-------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+
summary| Age| Total_Purchase| Account_Manager| Years| Num_Sites| Churn|
+-------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+
 count| 900| 900| 900| 900| 900| 900|
 mean|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|0.16666666666666666|
 stddev|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| 0.3728852122772358|
 min| 22.0| 100.0| 0| 1.0| 3.0| 0|
 max| 65.0| 18026.01| 1| 9.15| 14.0| 1|
+-------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+

In [0]:
data.columns

Out[7]: ['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [0]:
output = assembler.transform(data)

In [0]:
final_data = output.select('features','churn')

### Test Train Split

In [0]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

### Fit the model

In [0]:
from pyspark.ml.classification import LogisticRegression

In [0]:
lr_churn = LogisticRegression(labelCol='churn')

In [0]:
fitted_churn_model = lr_churn.fit(train_churn)

In [0]:
training_sum = fitted_churn_model.summary

In [0]:
training_sum.predictions.describe().show()

+-------+------------------+-------------------+
summary| churn| prediction|
+-------+------------------+-------------------+
 count| 617| 617|
 mean|0.1766612641815235| 0.1393841166936791|
 stddev|0.3816912173425678|0.34662792555315736|
 min| 0.0| 0.0|
 max| 1.0| 1.0|
+-------+------------------+-------------------+

### Evaluate results

Let's evaluate the results on the data set we were given (using the test data)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [0]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
 features|churn| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
[25.0,9672.03,0.0...| 0|[4.74946500683456...|[0.99141796385509...| 0.0|
[26.0,8787.39,1.0...| 1|[0.40491369390852...|[0.59986765329765...| 0.0|
[26.0,8939.61,0.0...| 0|[6.42290503416914...|[0.99837870108044...| 0.0|
[28.0,11128.95,1....| 0|[4.10971682190440...|[0.98385259648613...| 0.0|
[29.0,9617.59,0.0...| 0|[4.49641640533890...|[0.98897404888891...| 0.0|
[29.0,11274.46,1....| 0|[4.46532866276174...|[0.98862985189316...| 0.0|
[29.0,12711.15,0....| 0|[5.63997890068745...|[0.99645963534503...| 0.0|
[30.0,8403.78,1.0...| 0|[5.67836841891484...|[0.99659251814510...| 0.0|
[30.0,10960.52,1....| 0|[2.28396373323207...|[0.90754018540063...| 0.0|
[31.0,5304.6,0.0,...| 0|[3.15689950829300...|[0.95917972251223...| 0.0|
[31.0,5387.75,0.0...| 0|[2.23652730850396...|[0.90348205586540...| 0.0|
[31.0,10182.6,1.0...| 0|[4.69919266910964...|[0.99097948736077...| 0.0|
[31.0,12264.68,1....| 0|[3.50330314559713...|[0.97078160774844...| 0.0|
[32.0,6367.22,1.0...| 0|[2.68550884139642...|[0.93616611915849...| 0.0|
[32.0,7896.65,0.0...| 0|[3.27395037792057...|[0.96352426328326...| 0.0|
[32.0,12403.6,0.0...| 0|[5.89093830657789...|[0.99724323909708...| 0.0|
[32.0,12479.72,0....| 0|[4.89424279860774...|[0.99256609858952...| 0.0|
[33.0,5738.82,0.0...| 0|[4.00706100954361...|[0.98213808302695...| 0.0|
[33.0,7492.9,0.0,...| 0|[4.63457547982887...|[0.99038315342496...| 0.0|
[33.0,11370.28,1....| 0|[6.61730188549737...|[0.99866475171882...| 0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

### Using AUC

In [0]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [0]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [0]:
auc

Out[23]: 0.7618927635557348

### Predict new unlabeled data

The new_customers.csv file

In [0]:
final_lr_model = lr_churn.fit(final_data)

In [0]:
new_customers = spark.read.csv('/FileStore/tables/new_customers-1.csv',inferSchema=True,
                              header=True)

In [0]:
new_customers.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: string (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)

In [0]:
test_new_customers = assembler.transform(new_customers)

In [0]:
test_new_customers.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: string (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- features: vector (nullable = true)

In [0]:
final_results = final_lr_model.transform(test_new_customers)

In [0]:
final_results.select('Company','prediction').show()

+----------------+----------+
 Company|prediction|
+----------------+----------+
 King Ltd| 0.0|
 Cannon-Benson| 1.0|
Barron-Robertson| 1.0|
 Sexton-Golden| 1.0|
 Wood LLC| 0.0|
 Parks-Robbins| 1.0|
+----------------+----------+